In [1]:
import numpy as np
import tensorflow as tf
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

2025-06-09 15:37:26.836133: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749483447.126567      69 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749483447.209217      69 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
X = np.loadtxt("/kaggle/input/coffee-roasting-data-binary/input.csv", delimiter=",")
Y = np.loadtxt("/kaggle/input/coffee-roasting-data-binary/output.csv", delimiter=",")
Y = Y.reshape(-1,1)

In [3]:
Xt = np.tile(X,(1000,1))
Yt = np.tile(Y,(1000,1))   
print(Xt.shape, Yt.shape) 

(200000, 2) (200000, 1)


In [4]:
model = Sequential(
    [
        tf.keras.Input(shape=(2,)),
        Dense(2, activation='linear',  name = 'layer1'),
        Dense(2, activation='sigmoid', name = 'layer2'),
        Dense(1, activation='sigmoid', name = 'layer3')
     ]
)

2025-06-09 15:37:44.130423: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [5]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ layer1 (Dense)                       │ (None, 2)                   │               6 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer2 (Dense)                       │ (None, 2)                   │               6 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer3 (Dense)                       │ (None, 1)                   │               3 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15 (60.00 B)

 Trainable params: 15 (60.00 B)

 Non-trainable params: 0 (0.00 B)

In [6]:
import time
from tensorflow.keras.callbacks import Callback

class EpochTimer(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        self.start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        duration = (time.time() - self.start_time)*1000
        print(f"⏱️ Epoch {epoch + 1} took {duration:.1f} ms.")


In [7]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.1),
)

In [8]:
W1 = np.array([
    [-1.0, 5.0],
    [10.0, 7.0]] )
W2 = np.array([
    [-15.0, -1.0],
	[10.0, 7.0]])
W3 = np.array([
    [-12.0],
    [-15.0]])

b1 = np.array([-41.0, 22.0])
b2 = np.array([-28.0, -91.0])
b3 = np.array([31.0])

model.get_layer("layer1").set_weights([W1,b1])
model.get_layer("layer2").set_weights([W2,b2])
model.get_layer("layer3").set_weights([W3,b3])

In [9]:
start_time = time.time()
model.fit(
    Xt,Yt,  
    epochs = 10,
    batch_size = 25000,
    # steps_per_epoch = 8,
    shuffle = False,
    callbacks=[EpochTimer()],
)
end_time = time.time()
time_taken = (end_time - start_time)*1000
print(f"\nTotal time = {time_taken:.0f} ms\n")

Epoch 1/10
1/8 ━━━━━━━━━━━━━━━━━━━━ 3s 476ms/step - loss: 3.1581⏱️ Epoch 1 took 540.0 ms.
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 2.8239  
Epoch 2/10
1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 1.8461⏱️ Epoch 2 took 83.7 ms.
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.5999 
Epoch 3/10
1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.9885⏱️ Epoch 3 took 83.1 ms.
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.8851 
Epoch 4/10
1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.6636⏱️ Epoch 4 took 81.1 ms.
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6330 
Epoch 5/10
1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.5679⏱️ Epoch 5 took 82.8 ms.
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5585 
Epoch 6/10
1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.5378⏱️ Epoch 6 took 80.9 ms.
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5345 
Epoch 7/10
1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.5272⏱️ Epoch 7 took 82.6 ms.
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5260 
Epoch 8/10

In [10]:
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
W3, b3 = model.get_layer("layer3").get_weights()

print("W1:\n", W1, "\nb1:", b1)
print("\nW2:\n", W2, "\nb2:", b2)
print("\nW3:\n", W3, "\nb3:", b3)

W1:
 [[-1.  5.]
 [10.  7.]] 
b1: [-41.  22.]

W2:
 [[-15.  -1.]
 [ 10.   7.]] 
b2: [-28. -91.]

W3:
 [[-13.748726]
 [-16.74873 ]] 
b3: [29.25127]


In [11]:
Y_predict = model.predict(Xt);

6250/6250 ━━━━━━━━━━━━━━━━━━━━ 6s 931us/step


In [16]:
m = len(Yt)

accuracy = 0;        
for i in range(m):
    if((Y_predict[i] >= 0.5) == Yt[i]):
        accuracy += 100

accuracy /= m
print("Accuracy:",accuracy)

Accuracy: 78.5
